# Import & Load Childhood data (Under-One Year)

### Config

In [3]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [4]:
remote_path = ''
local_path = '../data/'

In [5]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

/Users/cnavarreteliz/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Load file

In [6]:
#Open unzipped file
df = pd.read_csv(local_path+'mortality.csv',delimiter=",")
df = df[['year','comuna_id','rate_comuna','rate_region','rate_country','count', 'age_range_id']]
df = df.astype({'year':'int','comuna_id':'int','rate_comuna':'float','rate_region':'float','rate_country':'float', 'age_range_id':'int'})
list(df)

['year',
 'comuna_id',
 'rate_comuna',
 'rate_region',
 'rate_country',
 'count',
 'age_range_id']

### Ingest

In [7]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'mortality', 'fact_mortality')

DROP TABLE IF EXISTS mortality.fact_mortality;
CREATE TABLE "mortality"."fact_mortality" (
"year" INTEGER,
  "comuna_id" INTEGER,
  "rate_comuna" REAL,
  "rate_region" REAL,
  "rate_country" REAL,
  "count" INTEGER,
  "age_range_id" INTEGER
)
COPY "mortality"."fact_mortality" ("year","comuna_id","rate_comuna","rate_region","rate_country","count","age_range_id") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Add Indices to foreign keys

In [9]:
engine.execute("""
CREATE INDEX fact_mortality_comuna_id_index 
ON mortality.fact_mortality (comuna_id)
""")

engine.execute("""
CREATE INDEX fact_mortality_year_index 
ON mortality.fact_mortality ("year");
""")

### Relation tables

In [20]:
d2 = download_file(remote_path,local_path,'age_range_id.csv')
print (inline_table_xml(d2, 'age_range_id', 'id', 'en'))

Already downloaded. Using: ../data/age_range_id.csv

<InlineTable alias="age_range_id">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Deaths under 7 days</Value>
      <Value column="es_description">Deaths under 7 days</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Deaths under 28 days</Value>
      <Value column="es_description">Deaths under 28 days</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Deaths 28 days to 11 months</Value>
      <Value column="es_description">Deaths 28 days to 11 months</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Deaths under 1 year</Value>
      <Value column="es_description">Deaths under 1 year